In [5]:
import datetime
import importlib.util
import sys
from tradingstrategy.client import Client
from tradeexecutor.utils.default_strategies import get_default_strategies_path
from tradeexecutor.strategy.execution_context import notebook_execution_context


from tradeexecutor.utils.default_strategies import get_default_strategies_path
from tradeexecutor.strategy.strategy_module import read_strategy_module

strategies_folder = get_default_strategies_path()
strategy_path = strategies_folder / "spot-and-short-momentum.py"

strategy_mod = read_strategy_module(strategy_path)

print(f"Backtesting for strategy {strategy_path}, engine version {strategy_mod.trading_strategy_engine_version}")

client = Client.create_jupyter_client()

# contains start_at, end_at backtesting range
universe_options = strategy_mod.get_universe_options()

strategy_universe = strategy_mod.create_trading_universe(
    datetime.datetime.utcnow(),
    client,
    notebook_execution_context,
    universe_options,
)

data_universe = strategy_universe.data_universe

Backtesting for strategy /Users/moo/code/ts/trade-executor/strategies/spot-and-short-momentum.py, engine version 0.3
Started Trading Strategy in Jupyter notebook environment, configuration is stored in /Users/moo/.tradingstrategy


## Explore momentum by weekday



In [6]:
import pandas as pd
from collections import defaultdict


def calculate_momentum_by_weekday(df, momentum_period: pd.TimeDelta, weekday: int) -> pd.Series:
    # Monday = 0 and Sunday = 6
    index = []
    values = []
    for date in df.index[df.index.weekday==weekday]:
        first = df[date + momentum_period]["close"]
        last = df[date]["open"]
        momentum = (last - first) / first 
        index.append(date)
        values.append(momentum)
    
    return pd.Series(values, index)
     

In [7]:
# Do ETH first
from tradingstrategy.chain import ChainId

pair = data_universe.pairs.get_pair_by_human_description((ChainId.polygon, "uniswap-v3", "WETH", "USDC"))
df = data_universe.candles.get_candles_by_pair(pair)
assert df is not None
momentums = pd.DataFrame()

for day in range(0, 7):
    day_name = calendar.day_name[day]
    print(f"Creating for {day_name}")
    momentums[day_name] = calculate_momentum_by_weekday(df, pd.Timedelta(days=-7), day)




Creating for 0
2022-07-04 00:00:00
2022-07-11 00:00:00
2022-07-18 00:00:00
2022-07-25 00:00:00
2022-08-01 00:00:00
2022-08-08 00:00:00
2022-08-15 00:00:00
2022-08-22 00:00:00
2022-08-29 00:00:00
2022-09-05 00:00:00
2022-09-12 00:00:00
2022-09-19 00:00:00
2022-09-26 00:00:00
2022-10-03 00:00:00
2022-10-10 00:00:00
2022-10-17 00:00:00
2022-10-24 00:00:00
2022-10-31 00:00:00
2022-11-07 00:00:00
2022-11-14 00:00:00
2022-11-21 00:00:00
2022-11-28 00:00:00
2022-12-05 00:00:00
2022-12-12 00:00:00
2022-12-19 00:00:00
2022-12-26 00:00:00
2023-01-02 00:00:00
2023-01-09 00:00:00
2023-01-16 00:00:00
2023-01-23 00:00:00
2023-01-30 00:00:00
2023-02-06 00:00:00
2023-02-13 00:00:00
2023-02-20 00:00:00
2023-02-27 00:00:00
2023-03-06 00:00:00
2023-03-13 00:00:00
2023-03-20 00:00:00
2023-03-27 00:00:00
2023-04-03 00:00:00
2023-04-10 00:00:00
2023-04-17 00:00:00
2023-04-24 00:00:00
2023-05-01 00:00:00
2023-05-08 00:00:00
2023-05-15 00:00:00
2023-05-22 00:00:00
2023-05-29 00:00:00
2023-06-05 00:00:00
2023-

In [8]:
# weekday -> period -> pair id

# periods = [6, 7, 8]
# pair_dfs = defaultdict(lambda: defaultdict(int))

# data_df = {}
# for pair in data_universe.pairs.get_all_pair_ids():
#     data_df[pair] = data_universe.candles.get_candles_by_pair(pair)

# for pair in data_universe.pairs.get_all_pair_ids():
#     data = data_df[pair]
#     for day in range(0, 7):
#         for period in periods:
#             pair_dfs[day][period][pair] = calculate_momentum_by_weekday(period, day)
